In [1]:
%run ./functions.ipynb

In [2]:
# create schema
execute_query("""
--DROP TABLE IF EXISTS openai002;
CREATE TABLE IF NOT EXISTS openai002 (
    id SERIAL PRIMARY KEY,
    reference TEXT,
    content TEXT,
    embedding vector(1000)
);
""")

In [3]:
def add_page(reference, content):
    embedding = generate_embedding(content, dimensions=1000)
    return execute_query(
        "INSERT INTO openai002(reference, content, embedding) VALUES (%s, %s, %s) RETURNING *", 
        (reference, content, embedding)
    )

In [4]:
def search(query, limit = 1):
    q1 = generate_embedding(query, dimensions=1000)
    #return execute_query(f"SELECT id, reference, content FROM openai002 ORDER BY embedding <-> '[{','.join(map(str, q1))}]'::vector LIMIT 1;")
    return execute_query("SELECT id, reference, content FROM openai002 ORDER BY embedding <-> %s::vector LIMIT %s", [q1, limit]);

In [5]:
add_page('page1', "create_user() - this function let you create a new user")
add_page('page1', "edit_user() - this function let you modify a user")
add_page('page1', "delete_user() - this function let you delete a user")

add_page('page2', "create_product() - this function let you create a new product")
add_page('page2', "edit_product() - this function let you modify a product")
add_page('page2', "delete_product() - this function let you delete a product")

[(6,
  'page2',
  'delete_product() - this function let you delete a product',
  '[0.03329421,0.01822527,-0.06567206,-0.02690518,-0.0073817433,-0.008762638,0.052995827,0.074428335,0.01466167,0.004457682,0.061446648,-0.06872658,-0.033523295,-0.017308915,0.022972494,0.02036343,0.0032040584,-0.0063381176,-0.03135968,0.047726788,0.012472601,0.014954394,0.014470763,0.017181644,0.024168845,0.030748779,0.033166938,-0.030494235,-0.0014230537,-0.041694123,-0.001988616,-0.030977868,0.03431238,0.0017849819,0.04003959,-0.024665205,0.06511207,0.031919677,-0.03543237,0.027236087,-0.04897405,-0.006579933,-0.017359823,-0.007884465,0.042534113,0.056101248,-0.06903203,0.019039806,0.016901646,-0.011556246,0.0664866,-0.05243583,0.0044672275,0.06429753,-0.042610478,0.015425297,0.051570386,0.034948736,0.058035776,0.0501704,-0.0021461146,-0.02665064,0.06302481,-0.012599872,-0.058341227,0.015183482,-0.00033746817,0.013121685,-0.03497419,0.03479601,-0.032632396,-0.021152513,-0.044621363,-0.008399915,0.00824082

In [6]:
print_rows(search("I want to create a new user"));

(1, 'page1', 'create_user() - this function let you create a new user')


In [7]:
print_rows(search("I want to change a product"));

(5, 'page2', 'edit_product() - this function let you modify a product')


In [8]:
print_rows(search("delete an thing from my inventory"));

(6, 'page2', 'delete_product() - this function let you delete a product')
